In [9]:
import pandas as pd
import numpy as np

from sklearn.metrics import classification_report, accuracy_score

In [10]:
# Performance measures

def prediction_measures(y_test,y_pred):
    performance = classification_report(y_test,y_pred,output_dict=True)
    return performance['depression'],performance['normal'],performance['accuracy']

# Support measures

def percentage_depression(pred):
    return np.sum(pred == 'depression') / len(pred)

def true_positives(pred, true):
    return np.sum((pred == 'depression') & (true == "depression"))

def true_negatives(pred, true):
    return np.sum((pred == 'normal') & (true == "normal"))

def true_positive_rate(pred, true):
    return true_positives(pred,true)/(true_positives(pred,true)+true_negatives(pred,true))

def true_negative_rate(pred, true):
    return true_negatives(pred,true)/(true_positives(pred,true)+true_negatives(pred,true))

# Complete measures

def statistical_parity(predsensitive,predother):
    return percentage_depression(predsensitive)/percentage_depression(predother)

def equal_opportunity(predsensitive, truesensitive, predother, trueother):
    return true_positive_rate(predsensitive,truesensitive)/true_positive_rate(predother,trueother)

def equalised_odds(predsensitive, truesensitive, predother, trueother):
    return (true_positive_rate(predsensitive,truesensitive)+true_negative_rate(predsensitive,truesensitive))/(true_positive_rate(predother,trueother)+true_negative_rate(predsensitive,truesensitive))

def equal_accuracy(predsensitive, truesensitive, predother, trueother):
    return accuracy_score(predsensitive,truesensitive)/accuracy_score(predother,trueother)

# All measurements

def all_measures(predsensitive, truesensitive, predother, trueother, name='test',single=False):
    score_dict = {}
    score_dict['test'] = name
    depression_performance, normal_performance ,accuracy = prediction_measures(predsensitive,truesensitive)
    score_dict['depression0precision'] = depression_performance['precision']
    score_dict['depression0recall'] = depression_performance['recall']
    score_dict['depression0f1'] = depression_performance['f1-score']
    score_dict['depression0support'] = depression_performance['support']
    score_dict['normal0precision'] = normal_performance['precision']
    score_dict['normal0recall'] = normal_performance['recall']
    score_dict['normal0f1'] = normal_performance['f1-score']
    score_dict['normal0support'] = normal_performance['support']
    score_dict['accuracy0'] = accuracy
    if single == False:
        depression_performance, normal_performance ,accuracy = prediction_measures(predother,trueother)
        score_dict['depression1precision'] = depression_performance['precision']
        score_dict['depression1recall'] = depression_performance['recall']
        score_dict['depression1f1'] = depression_performance['f1-score']
        score_dict['depression1support'] = depression_performance['support']
        score_dict['normal1precision'] = normal_performance['precision']
        score_dict['normal1recall'] = normal_performance['recall']
        score_dict['normal1f1'] = normal_performance['f1-score']
        score_dict['normal1support'] = normal_performance['recall']
        score_dict['accuracy1'] = accuracy
        score_dict['statisticalParity01'] = statistical_parity(predsensitive,predother)
        score_dict['equalOpportunity01'] = equal_opportunity(predsensitive, truesensitive, predother, trueother)
        score_dict['equalisedOdds'] = equalised_odds(predsensitive, truesensitive, predother, trueother)
        score_dict['equalAccuracy'] = equal_accuracy(predsensitive, truesensitive, predother, trueother)
    return score_dict

In [13]:


options = ['w2vec','blingfire','reweightedblingfire','nltk','reweightednltk','spacysm','reweightedspacysm','spacylg','reweightedspacylg','spacytrf','reweightedspacytrf']

for name in options:
    df = pd.read_json(f'data/predictionData/{name}Pred.json', lines=True, orient='records')

    pred_male = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), ['prediction']]
    pred_female = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), ['prediction']]
    y_testm = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), ['label']]
    y_testf = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), ['label']]

    score_dict = all_measures(pred_female,y_testf,pred_male,y_testm, name)

    try:
        df = pd.read_csv('data/results/testresults.csv')
    except:
        df = pd.DataFrame()

    df_new = pd.DataFrame(score_dict,index=[len(df)])

    df = pd.concat([df,df_new],ignore_index=True)

    df.to_csv('data/results/testresults.csv', index=False)

/Users/bram/anaconda3/envs/OZP/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/Users/bram/anaconda3/envs/OZP/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)
/Users/bram/anaconda3/envs/OZP/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass 